In [21]:
%matplotlib inline
%precision 16
import numpy
import math
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
import scipy.integrate as integrate

In [ ]:
# Solutions to Heston PDE using explicit scheme.
def S_xi(K, alpha, xi, S_max, S_min):
    c1 = numpy.arcsinh((S_max - K)/alpha)
    c2 = numpy.arcsinh((S_min - K)/alpha)
    
    return K + alpha * numpy.sinh(c1 * xi + c2 * (1 - xi))

def S_prime((K, alpha, xi, S_max, S_min):
    c1 = numpy.arcsinh((S_max - K)/alpha)
    c2 = numpy.arcsinh((S_min - K)/alpha)
            
    return alpha * (c1 - c2) * numpy.cosh(c2 * (xi - 1) - c1 * xi)

def S_dbl_prime((K, alpha, xi, S_max, S_min):
    c1 = numpy.arcsinh((S_max - K)/alpha)
    c2 = numpy.arcsinh((S_min - K)/alpha)
            
    return - alpha * (c1 - c2)**2 * numpy.sinh(c2 * (xi - 1) - c1 * xi)
            
def V_eta(beta, eta, V_max):
    d = numpy.arcsinh(V_max / beta)
    
    return beta * numpy.sinh(d * eta)
                
def V_prime(beta, eta, V_max):
    d = numpy.arcsinh(V_max / beta)
    
    return beta * d * numpy.cosh(d * eta)

def V_dbl_prime(beta, eta, V_max):
    d = numpy.arcsinh(V_max / beta)
    
    return beta * d**2 * numpy.sinh(d * eta)
                
def Heston_Explicit(M, N, T, L):
    
    return S


In [87]:
#def Heston_Integral(f, x)
   # return
def Heston_Integral(phi, K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, j):
    
    u_1 = 1.0 / 2.0
    u_2 = - 1.0 / 2.0
    a = kappa * theta
    rspi = rho * sigma * phi * 1j
    tau = T - t
    
    b_1 = kappa + _lambda - rho * sigma
    b_2 = kappa + _lambda
    
    d_1 = numpy.sqrt((rspi - b_1)**2 - sigma**2 * (2.0 * u_1 * phi * 1j - phi**2))
    d_2 = numpy.sqrt((rho * sigma * phi * 1j - b_2)**2 - sigma**2 * (2.0 * u_2 * phi * 1j - phi**2))
    
    g_1 = (b_1 - rspi + d_1) / (b_1 - rspi - d_1)
    g_2 = (b_2 - rspi + d_2) / (b_2 - rspi - d_2)
   
    D_1 = (b_1 - rspi + d_1) / sigma**2 * ((1.0 - numpy.exp(d_1 * tau)) / (1.0 - g_1 * numpy.exp(d_1 * r)))
    D_2 = (b_2 - rspi + d_2) / sigma**2 * ((1.0 - numpy.exp(d_2 * tau)) / (1.0 - g_2 * numpy.exp(d_2 * r)))
    
    C_1 = r * phi * 1j * tau + a / sigma**2 * ((b_1 - rspi + d_1) * tau - 2.0 * numpy.log((1.0 - g_1 * numpy.exp(d_1 * tau))/(1.0 - g_1)))
    C_2 = r * phi * 1j * tau + a / sigma**2 * ((b_2 - rspi + d_2) * tau - 2.0 * numpy.log((1.0 - g_2 * numpy.exp(d_2 * tau))/(1.0 - g_2)))   
    
    f_1 = numpy.exp(C_1 + D_1 * v + 1j * phi * numpy.log(S))
    f_2 = numpy.exp(C_2 + D_2 * v + 1j * phi * numpy.log(S))
    
    if(j==1):
        return numpy.real((numpy.exp(-1j * phi * numpy.log(K)) * f_1)/(1j * phi))
    elif(j==2):
        return numpy.real((numpy.exp(-1j * phi * numpy.log(K)) * f_2)/(1j * phi))

In [155]:
# The closed-form solution for Heston PDE
from scipy.integrate import quad
def Exact_Solution(K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda):
    
    P_1 = 0.5 + 1.0 / numpy.pi * quad(Heston_Integral, 0, 1000, args=(K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, 1))[0]
    P_2 = 0.5 + 1.0 / numpy.pi * quad(Heston_Integral, 0, 1000, args=(K, S, v, T, t, r, kappa, theta, sigma, rho, _lambda, 2))[0]
    
    Call = S * P_1 - K * numpy.exp(- r * (T - t)) * P_2
    return Call

/Users/Francis.Chen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: RuntimeWarning: divide by zero encountered in log
/Users/Francis.Chen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: RuntimeWarning: divide by zero encountered in log


In [212]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np

K = 1200.0
T = 0.125
t = 0.0
r = 0.01
kappa = 0.02
theta = 0.04
sigma = 0.01
rho = 0.5
_lambda = 0.0


fig = plt.figure()
ax = fig.gca(projection='3d')
v = np.arange(0, 1.5, 0.1)
s = np.arange(0, 2000, 100)
V, S = np.meshgrid(v, s)

z = np.array([Exact_Solution(K, s, v, T, t, r, kappa, theta, sigma, rho, _lambda) for v, s in zip(np.ravel(V), np.ravel(S))])
Z = z.reshape(V.shape)

ax.plot_surface(V, S, Z)
ax.set_zlim(0, 800)
ax.set_xlim(1.2, 0.0)
ax.set_ylim(0, 2000)
plt.show()


In [236]:
K = 100.0
S = 100.0
V = 0.02
T = 0.5
t = 0.0
r = 0.01
kappa = 0.4
theta = 0.2
sigma = 0.2
rho = 0.5
_lambda = 0.0
print Exact_Solution(K, S, V, T, t, r, kappa, theta, sigma, rho, _lambda)

6.1956088359
